In [ ]:
working_directory = '/users/ironstein/documents/projects working directory/handwriting recognition using neural networks on FPGA final year project/image processing/ipython notebooks/python C extension testing/mark3'

import os
import time
import graphlab as gl
gl.canvas.set_target('ipynb')

os.chdir(working_directory)

In [ ]:
%%file test.c

#include <stdio.h>

void for_loop_test(int n) {
    int i;
    for(i=0;i<n;i++);
}

###generating shared object for the above C file (test.so)


In [ ]:
os.system('cd ' + working_directory)
!clang -c test.c
!clang -o test.so -shared test.c
!file test.so

###python file to access the functions in test.so shared library

In [ ]:
%%file _test.py

import numpy
import ctypes

_test_ = numpy.ctypeslib.load_library('test','.')
_test_.for_loop_test.argtypes = [ctypes.c_int]
_test_.for_loop_test.restype = ctypes.c_void_p

def C_for_test(n) : 
    _test_.for_loop_test(n)

In [ ]:
def Python_for_test(n) : 
    for i in range(n) : 
        pass

###decorator for calculating time of execution of a function

In [ ]:
def time_this_function(function) : 
    try : 
        time
    except : 
        import time
    
    def wrapper_function(*args,**kwargs) : 
        start_time = time.time()
        function(*args,**kwargs)
        elapsed_time = (time.time() - start_time)*1000
#         print(elapsed_time,end=' ms\n')
        return(elapsed_time)
        
    return wrapper_function

###testing Python and C functions for comparing the performance

In [ ]:
import _test

@time_this_function
def C_test(n) : 
    _test.C_for_test(n)
    
@time_this_function
def Python_test(n) : 
    Python_for_test(n)

###time to compare the difference

In [ ]:
C_time_array = []
Python_time_array = []
count_array = []
i = 1
while(True) : 
    C_time_array.append(C_test(i))
    Python_time_array.append(Python_test(i))
    count_array.append(i)
    i *= 10
    if(i > 1000000000) : 
        break

###generate a CSV (comma seperated values) file for the recorded data, to be used in Graphlab Create

In [ ]:
def generate_csv(name='default.csv',**kwargs) : 
    csv_file = open(name,'w')
    
    keys = kwargs.keys()
    values = kwargs.values()
    
    for key in keys[:-1] : 
        csv_file.write(key),
        csv_file.write(','),
    csv_file.write(keys[-1])
    csv_file.write('\n')

    for i in range(len(values[0])) : 
        for j in range(len(values)-1) : 
            csv_file.write(str(values[j][i])),
            csv_file.write(','),
        csv_file.write(str(values[len(values)-1][i]))
        csv_file.write('\n')
        
    csv_file.close()

In [ ]:
generate_csv(name='test.csv',c_time = C_time_array,python_time = Python_time_array)
g = gl.SFrame('test.csv')
g

In [ ]:
g.show(view='Line Chart', x='c_time',y='python_time')